In [5]:
import csv

datafiles = {'avgstations': 'climatecentral/allstations.csv', 
             'daymet': "climatecentral/climatecentraldaymet.csv",
            'prism': 'climatecentral/climatecentralprism.csv',
            'base': 'climatecentral/stationresults.csv'}

data = {}
for datakey, df in datafiles.iteritems():
    with open(df, 'rb') as fin:
        tempdata = csv.DictReader(fin)
        for row in tempdata:
            theobj = data.get(row['placename'], {})
            for measure in ['uhitmin', 'uhitmax', 'uhitavg']:
                theobj[datakey + measure] = row[measure]
            data[row['placename']] = theobj
            


In [23]:
from bokeh.io import gridplot, output_notebook, show, output_file
from bokeh.models import CustomJS,TapTool, ColumnDataSource
from bokeh.plotting import figure
import pandas as pd
output_notebook()
output_file('climatecentral/populationscatter.html')

fullgrid = []

for measure in datafiles.keys():
    rowset = []
    for measure2 in datafiles.keys():
        temps = figure(width=200, plot_height=200, title=measure + ", " + measure2, 
                   tools="pan,wheel_zoom,box_zoom,reset,tap")
        measure1set = []
        measure2set = []
        for placename, values in data.iteritems():
            try:
                t1 = float(values[measure + 'uhitavg'])
                t2 = float(values[measure2 + 'uhitavg'])
            except Exception,e :
#                 print e
                continue
            else:
                measure1set.append(t1)
                measure2set.append(t2)

        
        source = ColumnDataSource(data=dict(
            x=measure1set,
            y=measure2set))

        
        temps.circle('x', 'y', color='blue', size=2, source=source, alpha=0.5)
#         temps.circle(newv1, newv2, size=2, color="navy", alpha=0.5)
        
#         taptool = temps.select(type=TapTool)
#         taptool.callback = callback
        
        rowset.append(temps)
    fullgrid.append(rowset)
p = gridplot(fullgrid)
show(p)
        

Loading BokehJS ...